In [ ]:
!pip install d2l
!pip install mxnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.8 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.2
    Uninstalling pyparsing-3.1.2:
      Successfully uninstalled pyparsing-3.1.2
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: matplotlib-inl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 10.8 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3


In [ ]:
from mxnet.gluon.data.vision import transforms
from mxnet.gluon.data.vision import CIFAR10
from mxnet import gluon, init, nd, autograd, context
from mxnet.gluon import nn
from d2l import mxnet as d2l


In [ ]:
def load_data_cifar10(batch_size, resize=None):
    transformer = []
    if resize:
        transformer += [transforms.Resize(resize)]
    transformer += [transforms.ToTensor()]
    transformer = transforms.Compose(transformer)
    cifar10_train = CIFAR10(train=True)
    cifar10_test = CIFAR10(train=False)
    train_iter = gluon.data.DataLoader(cifar10_train.transform_first(transformer), batch_size, shuffle=True, last_batch='discard')
    test_iter = gluon.data.DataLoader(cifar10_test.transform_first(transformer), batch_size, shuffle=False, last_batch='discard')
    return train_iter, test_iter


In [ ]:
net = nn.Sequential()
net.add(nn.Conv2D(channels=32, kernel_size=3, padding=1, activation='relu'))
net.add(nn.Conv2D(channels=64, kernel_size=3, padding=1, activation='relu'))
net.add(nn.MaxPool2D(pool_size=2, strides=2))
net.add(nn.Conv2D(channels=128, kernel_size=3, padding=1, activation='relu'))
net.add(nn.Conv2D(channels=128, kernel_size=3, padding=1, activation='relu'))
net.add(nn.MaxPool2D(pool_size=2, strides=2))
net.add(nn.Flatten())
net.add(nn.Dense(256, activation="relu"))
net.add(nn.Dropout(0.5))
net.add(nn.Dense(256, activation="relu"))
net.add(nn.Dropout(0.5))
net.add(nn.Dense(10))


In [ ]:
def evaluate_accuracy(data_iter, net, ctx):
    acc_sum, n = nd.array([0], ctx=ctx), 0
    for X, y in data_iter:
        X = X.astype('float32').as_in_context(ctx)
        y = y.astype('float32').as_in_context(ctx)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.size
    return acc_sum.asscalar() / n


In [ ]:
def train(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs):
    loss = gluon.loss.SoftmaxCrossEntropyLoss()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, batch_count = 0.0, 0.0, 0, 0
        for X, y in train_iter:
            X, y = X.astype('float32').as_in_context(ctx), y.astype('float32').as_in_context(ctx)
            with autograd.record():
                y_hat = net(X)
                l = loss(y_hat, y)
            l.backward()
            trainer.step(batch_size)
            train_l_sum += l.sum().asscalar()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().asscalar()
            n += y.size
            batch_count += 1
        test_acc = evaluate_accuracy(test_iter, net, ctx)
        print(f'epoch {epoch + 1}, loss {train_l_sum / n:.4f}, train acc {train_acc_sum / n:.3f}, test acc {test_acc:.3f}')


In [ ]:
ctx = d2l.try_gpu()
num_epochs, lr, wd, batch_size = 10, 0.001, 1e-4, 128
net.initialize(ctx=ctx, init=init.Xavier(), force_reinit=True)
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': lr, 'wd': wd})
train_iter, test_iter = load_data_cifar10(batch_size, resize=32)
train(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)


epoch 1, loss 1.7669, train acc 0.341, test acc 0.518
epoch 2, loss 1.2556, train acc 0.549, test acc 0.642
epoch 3, loss 1.0408, train acc 0.636, test acc 0.693
epoch 4, loss 0.9129, train acc 0.683, test acc 0.716
epoch 5, loss 0.8099, train acc 0.723, test acc 0.741


#api

In [ ]:
pip install fastapi
pip install uvicorn

In [ ]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
from mxnet import nd, gluon, image
from mxnet.gluon.data.vision import transforms
import numpy as np

app = FastAPI()

# Carregar o modelo treinado
ctx = mx.cpu()
net.load_parameters('cifar10.params', ctx=ctx)

# Transformação para preparar a imagem
transformer = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor()
])

# Labels do CIFAR-10
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    # Carregar a imagem
    img = image.imread(file.file)
    img = transformer(img)
    img = img.expand_dims(axis=0).as_in_context(ctx)

    # Realizar a inferência
    output = net(img)
    pred = nd.argmax(output, axis=1).astype('int')

    # Retornar a resposta
    response = {"category": labels[int(pred.asscalar())]}
    return JSONResponse(content=response)

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)
